Практический проект по написанию собственного парсера.
Цель: В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-кода и собрать себе свой собственный датасет.
По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).

Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.

В качестве потенциальных целей для парсинга можно взять какие-нибудь блоги, выкачать оттуда публикации, авторов, число просмотром и комментариев. Можно посмотреть новостные ресурсы и скачать новостные статьи с их мета-информацией. Или любой другой интересный вам сайт.

Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла)
Критерии оценки: Успешно собранные данные - 10 из 10!

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import datetime

from tqdm import tqdm_notebook

In [2]:
def ngs24newsarticle(link):
    now = datetime.datetime.now()
    page = requests.get(link)
    page = page.content
    page = BeautifulSoup(page, 'lxml')
    headline = page.find('h2', attrs={'itemprop':'headline'}).find('span').text
    altheadline = page.find('p', attrs={'itemprop':'alternativeHeadline'}).find('span').text
    txtnewsfull = page.find('div', attrs={'itemprop':'articleBody'}).text.replace('\xa0',' ')
    datenews = page.find('div', attrs={'itemprop':'datePublished'}).get('datetime')
    views = int(page.find('div', attrs={'title':'просмотры'}).text.replace('\xa0',''))
    comments = int(page.find('a', attrs={'title':'комментарии'}).text.replace('\xa0',''))
    rating = page.find('div', attrs={'itemprop':'aggregateRating'}).text.replace('оцените материал','').strip()
    rubric = page.find('div', attrs={'class':'F1cd'}).text.strip()
    tags = page.find('div', attrs={'class':'KBarf'}).findAll('span')
    tagsstr = ''
    for each in tags:
        tagsstr += '#'+each.text+', '
    return  {
        'parced' : now.strftime("%Y-%m-%d %H:%M:%S"),
        'link' : link,
        'datenews': datenews, 
        'headline': headline, 
        'altheadline': altheadline, 
        'txtnewsfull' : txtnewsfull,
        'views': views,
        'comments': comments,
        'rating' : rating,
        'rubric' : rubric,
        'tags' : tagsstr.strip()
        }


In [3]:
testlink = ['https://ngs24.ru/news/more//66484084/', 'https://ngs24.ru/news/more//66483727/']
for link in testlink:
    print(ngs24newsarticle(link))

{'parced': '2020-02-08 13:52:32', 'link': 'https://ngs24.ru/news/more//66484084/', 'datenews': '2020-02-07 13:41:04', 'headline': 'Главный прокурор края уходит в отставку', 'altheadline': 'Источник NGS24.RU в прокуратуре сообщает об отставке Михаила Савчина', 'txtnewsfull': 'Михаил Савчин возглавлял краевую прокуратуру с 2012 года Фото: krasproc.ruИсточник в прокуратуре Красноярска сообщил об отставке главного прокурора края Михаила Савчина. Официального ответа ведомства на момент написания новости получить пока не удалось.До своего назначения в Красноярский край Михаил Савчин работал прокурором в Чечне до 2012 года.Незадолго до отставки Савчин отстранил от работы прокурора Норильска Хаважа Гутаева, подозреваемого во взяточничестве, тем не менее Гутаев сохранил зарплату и доплаты за выслугу лет на момент отстранения. Ряд СМИ связывает отставку Савчина с Гутаевым, которого тот якобы покровительствовал.Прокуратура края под руководством Михаила Савчина также запомнилась скандалом с основа

In [4]:
def ngspagelinks(page_number):
    main_url = allngsnews
    news_page = requests.get(main_url+str(page_number))
    news_page = news_page.content
    news_page = BeautifulSoup(news_page, 'lxml')
    news_links = news_page.findAll('h2', attrs = {'class' : 'K7cb'})
    news_links = [link.find('a').get('href') for link in news_links]
    return news_links

In [5]:
allngsnews = 'https://ngs24.ru/news/all/?page'
maxpages = 20
ngsnews = pd.DataFrame(columns=['parced',
                            'link',
                            'datenews', 
                            'headline', 
                            'altheadline', 
                            'txtnewsfull' ,
                            'views',
                            'comments',
                            'rating',
                            'rubric',
                            'tags'])

for page in tqdm_notebook(range(maxpages)):
    news_links = ngspagelinks(page)
    news_links = ['https://ngs24.ru'+ link for link in news_links]
    for news_link in tqdm_notebook(news_links, leave=False):
        try:
            sub_page_data = ngs24newsarticle(news_link)
            ngsnews = ngsnews.append(sub_page_data, ignore_index=True)
            #time.sleep(0.2)
        except:
            pass


In [7]:
ngsnews.head().T

,0,1,2,3,4
parced,2020-02-08 13:52:56,2020-02-08 13:52:56,2020-02-08 13:52:58,2020-02-08 13:52:59,2020-02-08 13:53:01
link,https://ngs24.ru/news/more/66485668/,https://ngs24.ru/news/more/66485626/,https://ngs24.ru/news/more/66485617/,https://ngs24.ru/news/more/66485605/,https://ngs24.ru/news/more/66485599/
datenews,2020-02-08 12:42:26,2020-02-08 11:32:39,2020-02-08 10:42:55,2020-02-08 10:23:29,2020-02-08 09:34:17
headline,Красноярская активистка направила обращение в ...,Молодые красноярцы сняли квартиру и оборудовал...,С ж/д моста на Свободном оторвалась балка и на...,«Это грубое вторжение человека в экологию»: же...,Красноярец прошелся по капоту припаркованного ...
altheadline,Ответ аппарата правительства ее рассмешил,Им грозит 20 лет колонии,На месте работают сотрудники ГИБДД,Она предложила лечить его гомеопатией,Машина стояла припаркованной вплотную к стене
txtnewsfull,Лаборатория должна появиться в горах в 6 км от...,Найденная коробка со всякими веществами и один...,Балка опасно повисла над проезжей частью Фото:...,Наталья Толоконская — врач в Новосибирском гом...,Задуманное молодому человеку удалось осуществи...
views,703,1863,1905,1192,3181
comments,9,4,7,16,14
rating,ЛАЙК1СМЕХ1УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0,ЛАЙК0СМЕХ0УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0,ЛАЙК0СМЕХ0УДИВЛЕНИЕ1ГНЕВ2ПЕЧАЛЬ0,ЛАЙК2СМЕХ0УДИВЛЕНИЕ0ГНЕВ0ПЕЧАЛЬ0,ЛАЙК8СМЕХ3УДИВЛЕНИЕ1ГНЕВ2ПЕЧАЛЬ1
rubric,Город,Криминал,Происшествия,Здоровье Китайский коронавирус,Дороги и транспорт


In [9]:
ngsnews.to_csv('ngsnews.csv',sep=';')